# I. Introduction

## Scenario

### Problematic

A couple has been living in Vancouver, British Columbia in Canada for almost 10 years now. the husband just got offered a job in Manila, Philippines, but even if he has visited the country twice, that doesn't give him too much information on where they should settle. The couple likes to plan ahead at the best of their possibilities. they are in the mid 30's, do not have any kids, so they like to go out often, eat out, drink and so on, but they do not like driving.
Therefore, they are looking for a place that would be great for them.

### To be solved

The problematic is to find a list of activities in Manila, map of public transportation available and also a good idea of rentals and their prices.

### Interested audience

Whoever is about to relocate to Manila, Philippines. But that could apply to any other city in the world.

## Data Section

### Data needed

- fetching all the venues in Manila
- retrieving all the public transportation 
- getting an average price of rentals

### Methodology

Use of foursquare API to inquire all the venues within Manila, and get any additional informations to the couple so they could make a decision


In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
# Manila,Philippines
address = 'Manila, philippines'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manila are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Manila are 14.5906216, 120.9799696.


In [4]:
Manila_latitude=14.5906216
Manila_longitude=120.9799696

In [5]:
# The code was removed by Watson Studio for sharing.

In [20]:
LIMIT = 150 # limit of number of venues returned by Foursquare API
radius = 800 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Manila_latitude, 
   Manila_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=T1YFTS213UT2UDOPUW4FAKRFM5T1XOGG4MO2IGWOLEZZENSN&client_secret=DWVR1W4OICYNXF0FZX3LHYYEZ4BQAN3QSMRYOI3FD2N5BBB0&v=20200210&ll=14.5906216,120.9799696&radius=800&limit=150'

In [21]:
results = requests.get(url).json()
#results

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
MAnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
MAnearby_venues =MAnearby_venues.loc[:, filtered_columns]
# filter the category for each row
MAnearby_venues['venue.categories'] = MAnearby_venues.apply(get_category_type, axis=1)
# clean columns
MAnearby_venues.columns = [col.split(".")[-1] for col in MAnearby_venues.columns]

MAnearby_venues.shape

(78, 4)

In [24]:
# Venues of Manila
MAnearby_venues.head()

,name,categories,lat,lng
0,The Bayleaf Hotel,Hotel,14.589945,120.978808
1,Adobo Connection,Filipino Restaurant,14.590246,120.982349
2,Sky Deck,Roof Deck,14.589915,120.978756
3,The French Baker,Bakery,14.590214,120.983140
4,Silahis Art and Artifacts,Antique Shop,14.587427,120.977411


In [25]:
latitude=14.5906216
longitude=120.9799696
# create map of Manila places  using latitude and longitude values
map_MA = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(MAnearby_venues['lat'], MAnearby_venues['lng'], MAnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_MA)  
    
map_MA

As we can see, mostly all the venues are located within Manila intramuros. However, a certain amount of venues are also located inside the SM city Manila, which is a huge mall, very popular.

Al the public transportation are displayed too, which will help a lot the couple to make their decision to choose which location where to look for a rental

In [26]:
! pip install requests
! pip install html5lib
! pip install bs4

  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [27]:
URL = "https://www.lamudi.com.ph/metro-manila/house/rent/"
r = requests.get(URL) 
print(r.content) 

b'\n<!DOCTYPE html>\n<html lang="en-PH" dir="ltr">\n<head>\n    <link rel="icon" type="image/vnd.microsoft.icon" href="/favicon.ico">\n    <link rel="preconnect" href="https://www.google-analytics.com">\n<link rel="preconnect" href="https://www.googleadservices.com">\n<link rel="preconnect" href="https://www.google.com">\n<link rel="preconnect" href="https://googleads.g.doubleclick.net">\n<link rel="preconnect" href="https://stats.g.doubleclick.net">\n<link rel="preconnect" href="https://www.googletagmanager.com">\n<link rel="preconnect" href="https://connect.facebook.net">\n<link rel="preconnect" href="https://adservice.google.com">\n<link rel="preconnect" href="https://securepubads.g.doubleclick.net">\n<link rel="preconnect" href="https://bam.nr-data.net">\n<link rel="preconnect" href="https://js-agent.newrelic.com">\n<link rel="preconnect" href="https://dev.visualwebsiteoptimizer.com">\n<link rel="preconnect" href="https://adservice.google.de">\n<link rel="preconnect" href="https://

In [30]:
!pip install selenium

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [34]:
from lxml import html
import requests

In [37]:
page = requests.get('https://www.tripadvisor.ca/ShowTopic-g298573-i3261-k6170018-Condo_rentals_by_month-Manila_Metro_Manila_Luzon.html')
tree = html.fromstring(page.content)

In [33]:
df_condos = pd.read_html('https://www.tripadvisor.ca/ShowTopic-g298573-i3261-k6170018-Condo_rentals_by_month-Manila_Metro_Manila_Luzon.html')

df_condos.head()

ValueError: No tables found